In [ ]:
import mysql.connector

# Connect to MySQL
def create_connection():
    try:
        conn = mysql.connector.connect(
            host="localhost",     
            user="root",          
            passwd="ashu",       
            database="glasses"      
        )
        if conn.is_connected():
            print("Connected to MySQL database")
        return conn
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

# Helper functions
def signup(cursor, conn):
    username = input("Enter a new username: ")
    password = input("Enter a new password: ")
    
    # Insert new user into the database
    cursor.execute("INSERT INTO Customer (username, password) VALUES (%s, %s)", (username, password))
    conn.commit()
    print("Signup successful! Please login to continue.")

def login(cursor):
    username = input("Enter your username: ")
    password = input("Enter your password: ")
    
    cursor.execute("SELECT * FROM Customer WHERE username=%s AND password=%s", (username, password))
    user = cursor.fetchone()
    
    if user:
        print(f"Welcome, {username}!")
        return user[0]  # Returning the customer_id
    else:
        print("Invalid login credentials!")
        return None

def show_products(cursor, category):
    if category == 'Men':
        cursor.execute("SELECT * FROM Men")
    else:
        cursor.execute("SELECT * FROM Women")
    
    products = cursor.fetchall()
    
    print(f"\nAvailable {category} products:")
    for product in products:
        print(f"ID: {product[0]} | Brand: {product[1]} | Frame Type: {product[2]} | Color: {product[3]} | Price: ${product[4]}")

def place_order(cursor, conn, customer_id, category):
    product_id = int(input("Enter the product ID to order: "))
    
    cursor.execute("INSERT INTO Orders (customer_id, brand_id, category) VALUES (%s, %s, %s)", (customer_id, product_id, category))
    conn.commit()
    print("Order placed successfully!")

def continue_shopping():
    return input("Do you want to continue shopping? (yes/no): ").lower() == 'yes'

# Main logic
def main():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        
        print("Welcome to the Glasses Shop!")
        
        choice = input("Would you like to sign up or login? (signup/login): ").lower()
        
        if choice == 'signup':
            signup(cursor, conn)
        
        customer_id = None
        while not customer_id:
            customer_id = login(cursor)
        
        while True:
            gender = input("Select category: Men or Women: ").capitalize()
            if gender not in ['Men', 'Women']:
                print("Invalid category. Please choose again.")
                continue
            
            show_products(cursor, gender)
            place_order(cursor, conn, customer_id, gender)
            
            if not continue_shopping():
                print("Thank you for shopping with us! Goodbye.")
                break

        # Close connection
        cursor.close()
        conn.close()

# Run the application
if __name__ == "__main__":
    main()
